In [5]:
import torch, torchvision, numpy as np, cv2

print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
print("NumPy:", np.__version__)
print("OpenCV:", cv2.__version__)


Torch: 2.3.1+cu121
Torchvision: 0.18.1+cu121
CUDA available: True
NumPy: 1.26.4
OpenCV: 4.12.0


# Start Code

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import json
import cv2
import numpy as np
import random

In [ ]:
# check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Torch Dataset

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)
        
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])  
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)
        
        kps[::2]* 224.0 / w #adjust x coordinates keypoints
        kps[1::2]* 224.0 / h #adjust y coordinates keypoints
        
        return img, kps

In [ ]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Create a Model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)  #replace last layer for 14 keypoints (x,y)

In [ ]:
model = model.to(device)

# Train model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs, kps = imgs.to(device), kps.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, Loss: {loss.item()}")

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")